In [1]:
import requests
import pandas as pd

In [9]:
accept = "application/json"
api_key = 'FaCbyvJhMewTCPyeDKQwBNBAlaV2Oj32'  
user_agent = ""  # leave this blank

HEADERS = {"accept": accept, "apiKey": api_key, "user-agent": user_agent}


def get_provider(npi: str) -> dict:
    url = f"https://api.trillianthealth.com/v1/providers/{npi}"
    response = requests.get(url, headers=HEADERS)
    return response.json()


def pprint_result(result: dict) -> None:
    from pprint import pprint

    pprint(result, sort_dicts=False)

In [11]:
npi = "1124447073"

pprint_result(get_provider(npi))

{'npi': '1124447073',
 'active': True,
 'name': {'last': 'Diulio',
          'first': 'Jonathon',
          'middle': 'James',
          'suffix': None,
          'full': 'Jonathon James Diulio'},
 'credential': 'MD',
 'gender': 'MALE',
 'estimatedAge': 38,
 'affiliatedOrganizations': {'items': [{'name': 'County of San Joaquin'}]},
 'affiliatedPractices': {'total': 3,
                         'items': [{'percentOfVisits': 0.522017333102234,
                                    'address': {'street': '500 W Hospital Rd',
                                                'city': 'French Camp',
                                                'state': 'CA',
                                                'zipCode': '95231',
                                                'county': 'SAN JOAQUIN'},
                                    'latLong': {'lat': 37.88617,
                                                'long': -121.28308}}]},
 'patientPanel': {'age': {'median': 38,
                       

In [20]:
# url = f"https://api.trillianthealth.com/v1/providers/{npi}"
url='https://api.stable.uaap.trillianthealth.com/api/provider-directory/providers:search?pageSize=100&pageNumber=0'
response = requests.get(url, headers=HEADERS)


In [23]:
response = requests.post(url, json=payload, headers=HEADERS)
response.raise_for_status()
response.json()

HTTPError: 404 Client Error: Not Found for url: https://api.trillianthealth.com/v1/organizations?page=1&pageSize=10

In [ ]:
'FaCbyvJhMewTCPyeDKQwBNBAlaV2Oj32'

In [ ]:
{
  "openapi" : "3.0.3",
  "info" : {
    "title" : "Provider Directory API"
  },
  "servers" : [ {
    "url" : "https://api.trillianthealth.com/",
    "description" : "Live API"
  } ],
  "paths" : {
    "/v1/providers/{npi}" : {
      "get" : {
        "tags" : [ ],
        "summary" : "Lookup a provider by npi",
        "parameters" : [ {
          "name" : "npi",
          "in" : "path",
          "schema" : {
            "format" : "int64",
            "type" : "integer"
          },
          "required" : true,
          "deprecated" : false
        } ],
        "responses" : {
          "200" : {
            "description" : "Success",
            "content" : {
              "application/json" : {
                "schema" : {
                  "$ref" : "#/components/schemas/ProviderResponse"
                }
              }
            }
          }
        },
        "deprecated" : false
      }
    },
    "/v1/organizations" : {
      "get" : {
        "tags" : [ ],
        "summary" : "Search organizations",
        "parameters" : [ {
          "name" : "page",
          "in" : "query",
          "schema" : {
            "format" : "int32",
            "type" : "integer",
            "default" : 0
          },
          "required" : false,
          "deprecated" : false
        }, {
          "name" : "pageSize",
          "in" : "query",
          "schema" : {
            "format" : "int32",
            "type" : "integer",
            "default" : 10,
            "maximum" : 10,
            "exclusiveMaximum" : false
          },
          "required" : false,
          "deprecated" : false
        }, {
          "name" : "npi",
          "in" : "query",
          "schema" : {
            "type" : "string",
            "nullable" : true
          },
          "required" : false,
          "deprecated" : false
        }, {
          "name" : "name",
          "in" : "query",
          "schema" : {
            "type" : "string",
            "nullable" : true
          },
          "required" : false,
          "deprecated" : false
        } ],
        "responses" : {
          "200" : {
            "description" : "Success",
            "content" : {
              "application/json" : {
                "schema" : {
                  "$ref" : "#/components/schemas/OrganizationPagingResponse"
                }
              }
            }
          }
        },
        "deprecated" : false
      }
    }
  },
  "components" : {
    "schemas" : {
      "ProviderResponse" : {
        "properties" : {
          "active" : {
            "type" : "boolean"
          },
          "affiliatedOrganizations" : {
            "$ref" : "#/components/schemas/AffiliatedOrganizations"
          },
          "affiliatedPractices" : {
            "$ref" : "#/components/schemas/AffiliatedPractices"
          },
          "credential" : {
            "type" : "string",
            "nullable" : true
          },
          "estimatedAge" : {
            "format" : "int32",
            "type" : "integer",
            "nullable" : true
          },
          "gender" : {
            "type" : "string",
            "nullable" : true
          },
          "medicalSchool" : {
            "$ref" : "#/components/schemas/MedicalSchool"
          },
          "name" : {
            "$ref" : "#/components/schemas/Name"
          },
          "npi" : {
            "type" : "string"
          },
          "patientPanel" : {
            "$ref" : "#/components/schemas/PatientPanel"
          },
          "primarySpecialty" : {
            "$ref" : "#/components/schemas/ProviderSpecialty"
          }
        },
        "required" : [ "npi", "active", "name", "credential", "gender", "estimatedAge", "affiliatedOrganizations", "affiliatedPractices", "patientPanel", "primarySpecialty", "medicalSchool" ],
        "type" : "object"
      },
      "AffiliatedOrganizations" : {
        "properties" : {
          "items" : {
            "items" : {
              "$ref" : "#/components/schemas/AffiliatedOrganization"
            },
            "type" : "array"
          }
        },
        "required" : [ "items" ],
        "type" : "object"
      },
      "AffiliatedOrganization" : {
        "properties" : {
          "name" : {
            "type" : "string",
            "nullable" : true
          }
        },
        "required" : [ "name" ],
        "type" : "object"
      },
      "AffiliatedPractices" : {
        "properties" : {
          "items" : {
            "items" : {
              "$ref" : "#/components/schemas/AffiliatedPractice"
            },
            "type" : "array"
          },
          "total" : {
            "format" : "int64",
            "type" : "integer"
          }
        },
        "required" : [ "total", "items" ],
        "type" : "object"
      },
      "AffiliatedPractice" : {
        "properties" : {
          "address" : {
            "$ref" : "#/components/schemas/Address"
          },
          "latLong" : {
            "$ref" : "#/components/schemas/LatLong"
          },
          "percentOfVisits" : {
            "format" : "double",
            "type" : "number",
            "nullable" : true
          }
        },
        "required" : [ "percentOfVisits", "address", "latLong" ],
        "type" : "object"
      },
      "Address" : {
        "properties" : {
          "city" : {
            "type" : "string",
            "nullable" : true
          },
          "county" : {
            "type" : "string",
            "nullable" : true
          },
          "state" : {
            "type" : "string",
            "nullable" : true
          },
          "street" : {
            "type" : "string",
            "nullable" : true
          },
          "zipCode" : {
            "type" : "string",
            "nullable" : true
          }
        },
        "required" : [ "street", "city", "state", "zipCode", "county" ],
        "type" : "object"
      },
      "LatLong" : {
        "properties" : {
          "lat" : {
            "format" : "double",
            "type" : "number",
            "nullable" : true
          },
          "long" : {
            "format" : "double",
            "type" : "number",
            "nullable" : true
          }
        },
        "required" : [ "lat", "long" ],
        "type" : "object"
      },
      "MedicalSchool" : {
        "properties" : {
          "graduationYear" : {
            "type" : "string",
            "nullable" : true
          },
          "name" : {
            "type" : "string",
            "nullable" : true
          }
        },
        "required" : [ "name", "graduationYear" ],
        "type" : "object"
      },
      "Name" : {
        "properties" : {
          "first" : {
            "type" : "string",
            "nullable" : true
          },
          "full" : {
            "type" : "string",
            "nullable" : true
          },
          "last" : {
            "type" : "string",
            "nullable" : true
          },
          "middle" : {
            "type" : "string",
            "nullable" : true
          },
          "suffix" : {
            "type" : "string",
            "nullable" : true
          }
        },
        "required" : [ "last", "first", "middle", "suffix" ],
        "type" : "object"
      },
      "PatientPanel" : {
        "properties" : {
          "age" : {
            "$ref" : "#/components/schemas/PatientPanelAge"
          },
          "gender" : {
            "$ref" : "#/components/schemas/PatientPanelGender"
          }
        },
        "required" : [ "age", "gender" ],
        "type" : "object"
      },
      "PatientPanelAge" : {
        "properties" : {
          "median" : {
            "format" : "int32",
            "type" : "integer",
            "nullable" : true
          },
          "percent0to19" : {
            "format" : "double",
            "type" : "number",
            "nullable" : true
          },
          "percent20to44" : {
            "format" : "double",
            "type" : "number",
            "nullable" : true
          },
          "percent45to64" : {
            "format" : "double",
            "type" : "number",
            "nullable" : true
          },
          "percent65to84" : {
            "format" : "double",
            "type" : "number",
            "nullable" : true
          },
          "percent85Plus" : {
            "format" : "double",
            "type" : "number",
            "nullable" : true
          }
        },
        "required" : [ "median", "percent0to19", "percent20to44", "percent45to64", "percent65to84", "percent85Plus" ],
        "type" : "object"
      },
      "PatientPanelGender" : {
        "properties" : {
          "percentFemale" : {
            "format" : "double",
            "type" : "number",
            "nullable" : true
          },
          "percentMale" : {
            "format" : "double",
            "type" : "number",
            "nullable" : true
          }
        },
        "required" : [ "percentMale", "percentFemale" ],
        "type" : "object"
      },
      "ProviderSpecialty" : {
        "properties" : {
          "code" : {
            "type" : "string",
            "nullable" : true
          },
          "description" : {
            "type" : "string",
            "nullable" : true
          }
        },
        "required" : [ "code", "description" ],
        "type" : "object"
      },
      "OrganizationPagingResponse" : {
        "properties" : {
          "items" : {
            "items" : {
              "$ref" : "#/components/schemas/OrganizationResponse"
            },
            "type" : "array"
          },
          "page" : {
            "format" : "int32",
            "type" : "integer"
          },
          "pageSize" : {
            "format" : "int32",
            "type" : "integer"
          },
          "total" : {
            "format" : "int32",
            "type" : "integer"
          }
        },
        "required" : [ "page", "pageSize", "total", "items" ],
        "type" : "object"
      },
      "OrganizationResponse" : {
        "properties" : {
          "address" : {
            "$ref" : "#/components/schemas/Address"
          },
          "associatedNpis" : {
            "items" : {
              "$ref" : "#/components/schemas/OrganizationNpiAndTaxonomy"
            },
            "type" : "array"
          },
          "location" : {
            "$ref" : "#/components/schemas/LatLong"
          },
          "name" : {
            "type" : "string"
          },
          "type" : {
            "type" : "string",
            "nullable" : true
          }
        },
        "required" : [ "name", "type", "address", "location", "associatedNpis" ],
        "type" : "object"
      },
      "OrganizationNpiAndTaxonomy" : {
        "properties" : {
          "npi" : {
            "type" : "string"
          },
          "taxonomy" : {
            "$ref" : "#/components/schemas/OrganizationTaxonomy"
          }
        },
        "required" : [ "npi", "taxonomy" ],
        "type" : "object"
      },
      "OrganizationTaxonomy" : {
        "properties" : {
          "code" : {
            "type" : "string",
            "nullable" : true
          },
          "description" : {
            "type" : "string",
            "nullable" : true
          }
        },
        "required" : [ "code", "description" ],
        "type" : "object"
      }
    },
    "securitySchemes" : {
      "ApiKey" : {
        "in" : "header",
        "name" : "apiKey",
        "type" : "apiKey"
      }
    }
  },
  "security" : [ {
    "ApiKey" : [ ]
  } ],
  "tags" : [ ]
}